# Versioned Artifacts System Quick Start

In this notebook, we will learn the best practice to deploy versioned artifacts and use alias to perform blue / green or canary deployment.

**Concenpts**

- **Artifact**: An artifact is an item produced or used during the software development process. In the context of the ``versioned`` Python library, the content of an artifact is always a binary object. For deployment purposes, if you need to produce multiple files, you can create a compressed archive file (e.g., ``*.tar.gz``) that includes all necessary files.
- **Version**: A version refers to a specific revision of an artifact's content. It is crucial that artifact versions remain **immutable** to ensure deterministic deployment. Versions are represented by auto-incremented numbers (e.g., 1, 2, 3, ...). In the ``versioned`` Python library, a hard limit of 999,999 revisions is imposed. It's important to note that ``LATEST`` is a special version that remains immutable. Each time an artifact is updated, it automatically updates the ``LATEST`` version.
- **Alias**: An alias is a pointer associated with a specific artifact version. In addition to the primary version, you can also map an alias to a secondary version. This capability allows for splitting invocation requests between two versions, providing flexibility in managing deployments.

**Deployment Workflow**

- In the development phase, you can continuously update the artifact using the ``put_artifact`` API, which only affects the ``LATEST`` version.
- When you are ready to release, utilize the ``publish_artifact_version`` API to create an immutable snapshot of your ``LATEST`` version.
- To serve production traffic, it is recommended to use the ``put_alias`` API to create a ``LIVE`` alias that points to the LATEST version.
- In case you encounter any issues with the new release, you always have the option to reassociate the ``LIVE`` alias with a historical artifact version. This approach is commonly known as a ``Blue/Green deployment``. In this method, the ``LATEST`` version represents the ``Green`` environment (new release), while the historical version serves as the ``Blue`` environment (a stable working environment that can be rolled back to if needed).
- For canary deployments, you can map the ``LIVE`` alias to a secondary version and initially route only 10% of the traffic to the secondary version. Gradually, you can increase the weight of the secondary version, allowing it to serve a larger share of the traffic. Eventually, the secondary version will become the primary version and handle 100% of the traffic.

## Import the Library

In [1]:
from versioned.api import Repository
from boto_session_manager import BotoSesManager
from rich import print as rprint

bsm = BotoSesManager(profile_name="awshsh_app_dev_us_east_1")
repo = Repository(
    aws_region=bsm.aws_region,
    s3_bucket=f"{bsm.aws_account_id}-{bsm.aws_region}-artifacts",
    s3_prefix="versioned-artifacts",
    dynamodb_table_name="versioned-artifacts",
)

repo.bootstrap(bsm=bsm)

name = "deploy"

# uncommend this line to purge all artifacts version and their database records
repo.purge_artifact(bsm=bsm, name=name)

## Create Your First Artifact

An artifact has to have a unique name.

In [2]:
artifact = repo.put_artifact(bsm=bsm, name=name, content=b"v1")
rprint(artifact)
print(f"preview on S3: {artifact.s3path.console_url}")
print(f"preview artifact content: {artifact.get_content(bsm=bsm)}")

Artifact(
    name='deploy',
    version='LATEST',
    update_at=datetime.datetime(2023, 7, 6, 16, 50, 36, 88002, tzinfo=datetime.timezone.utc),
    s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/LATEST',
    sha256='3bfc269594ef649228e9a74bab00f042efc91d5acc6fbee31a382e80d42388fe'
)

preview on S3: https://console.aws.amazon.com/s3/object/807388292768-us-east-1-artifacts?prefix=versioned-artifacts/deploy/LATEST
preview artifact content: b'v1'


In [3]:
artifact = repo.get_artifact_version(bsm=bsm, name=name)
rprint(artifact)

Artifact(
    name='deploy',
    version='LATEST',
    update_at=datetime.datetime(2023, 7, 6, 16, 50, 36, 88002, tzinfo=datetime.timezone.utc),
    s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/LATEST',
    sha256='3bfc269594ef649228e9a74bab00f042efc91d5acc6fbee31a382e80d42388fe'
)

## Publish a New Artifact Version from the ``LATEST`` Version

In [4]:
artifact = repo.publish_artifact_version(bsm=bsm, name=name)
rprint(artifact)
print(f"preview on S3: {artifact.s3path.console_url}")
print(f"preview artifact content: {artifact.get_content(bsm=bsm)}")

Artifact(
    name='deploy',
    version='1',
    update_at=datetime.datetime(2023, 7, 6, 16, 50, 37, 576841, tzinfo=datetime.timezone.utc),
    s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000001',
    sha256='3bfc269594ef649228e9a74bab00f042efc91d5acc6fbee31a382e80d42388fe'
)

preview on S3: https://console.aws.amazon.com/s3/object/807388292768-us-east-1-artifacts?prefix=versioned-artifacts/deploy/000001
preview artifact content: b'v1'


In [5]:
artifact = repo.get_artifact_version(bsm=bsm, name=name, version=1)
rprint(artifact)

Artifact(
    name='deploy',
    version='1',
    update_at=datetime.datetime(2023, 7, 6, 16, 50, 37, 576841, tzinfo=datetime.timezone.utc),
    s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000001',
    sha256='3bfc269594ef649228e9a74bab00f042efc91d5acc6fbee31a382e80d42388fe'
)

In [6]:
artifact_list = repo.list_artifact_versions(bsm=bsm, name=name)
rprint(artifact_list)

[
    Artifact(
        name='deploy',
        version='LATEST',
        update_at=datetime.datetime(2023, 7, 6, 16, 50, 36, 88002, tzinfo=datetime.timezone.utc),
        s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/LATEST',
        sha256='3bfc269594ef649228e9a74bab00f042efc91d5acc6fbee31a382e80d42388fe'
    ),
    Artifact(
        name='deploy',
        version='1',
        update_at=datetime.datetime(2023, 7, 6, 16, 50, 37, 576841, tzinfo=datetime.timezone.utc),
        s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000001',
        sha256='3bfc269594ef649228e9a74bab00f042efc91d5acc6fbee31a382e80d42388fe'
    )
]

## Update the ``LATEST`` Version and Publish a New Version

Each time an artifact is updated, it automatically updates the ``LATEST`` version.

In [7]:
artifact = repo.put_artifact(bsm=bsm, name=name, content=b"v2")
rprint(artifact)
print(f"preview on S3: {artifact.s3path.console_url}")
print(f"preview artifact content: {artifact.get_content(bsm=bsm)}")

Artifact(
    name='deploy',
    version='LATEST',
    update_at=datetime.datetime(2023, 7, 6, 16, 50, 38, 115081, tzinfo=datetime.timezone.utc),
    s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/LATEST',
    sha256='fb04dcb6970e4c3d1873de51fd5a50d7bb46b3383113602665c350ec40b5f990'
)

preview on S3: https://console.aws.amazon.com/s3/object/807388292768-us-east-1-artifacts?prefix=versioned-artifacts/deploy/LATEST
preview artifact content: b'v2'


In [8]:
artifact = repo.publish_artifact_version(bsm=bsm, name=name)
rprint(artifact)
print(f"preview on S3: {artifact.s3path.console_url}")
print(f"preview artifact content: {artifact.get_content(bsm=bsm)}")

Artifact(
    name='deploy',
    version='2',
    update_at=datetime.datetime(2023, 7, 6, 16, 50, 38, 484012, tzinfo=datetime.timezone.utc),
    s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000002',
    sha256='fb04dcb6970e4c3d1873de51fd5a50d7bb46b3383113602665c350ec40b5f990'
)

preview on S3: https://console.aws.amazon.com/s3/object/807388292768-us-east-1-artifacts?prefix=versioned-artifacts/deploy/000002
preview artifact content: b'v2'


In [9]:
artifact_list = repo.list_artifact_versions(bsm=bsm, name=name)
rprint(artifact_list)

[
    Artifact(
        name='deploy',
        version='LATEST',
        update_at=datetime.datetime(2023, 7, 6, 16, 50, 38, 115081, tzinfo=datetime.timezone.utc),
        s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/LATEST',
        sha256='fb04dcb6970e4c3d1873de51fd5a50d7bb46b3383113602665c350ec40b5f990'
    ),
    Artifact(
        name='deploy',
        version='2',
        update_at=datetime.datetime(2023, 7, 6, 16, 50, 38, 484012, tzinfo=datetime.timezone.utc),
        s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000002',
        sha256='fb04dcb6970e4c3d1873de51fd5a50d7bb46b3383113602665c350ec40b5f990'
    ),
    Artifact(
        name='deploy',
        version='1',
        update_at=datetime.datetime(2023, 7, 6, 16, 50, 37, 576841, tzinfo=datetime.timezone.utc),
        s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000001',
        sha256='3bfc269594ef649228e9a74bab00f042efc91d5acc6fbee31a382e80d42388fe'
    )
]

## Create an Alias

In [10]:
alias = "LIVE"

ali = repo.put_alias(bsm=bsm, name=name, alias=alias)
rprint(ali)

Alias(
    name='deploy',
    alias='LIVE',
    version='LATEST',
    secondary_version=None,
    secondary_version_weight=None,
    version_s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/LATEST',
    secondary_version_s3uri=None
)

## Update an Alias for Canary Deployment

In [11]:
ali = repo.put_alias(bsm=bsm, name=name, alias=alias, version=1, secondary_version=2, secondary_version_weight=10)
rprint(ali)
rprint(ali.s3path_version.console_url)

Alias(
    name='deploy',
    alias='LIVE',
    version='1',
    secondary_version='2',
    secondary_version_weight=10,
    version_s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000001',
    secondary_version_s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000002'
)

https://console.aws.amazon.com/s3/object/807388292768-us-east-1-artifacts?prefix=versioned-artifacts/deploy/000001

In [12]:
ali = repo.put_alias(bsm=bsm, name=name, alias=alias, version=1, secondary_version=2, secondary_version_weight=50)
print(f"preview primary version on S3: {ali.s3path_version.console_url}")
print(f"preview primary artifact version content: {ali.get_version_content(bsm=bsm)}")
print(f"preview secondary version on S3: {ali.s3path_secondary_version.console_url}")
print(f"preview secondary artifact version content: {ali.get_secondary_version_content(bsm=bsm)}")

preview primary version on S3: https://console.aws.amazon.com/s3/object/807388292768-us-east-1-artifacts?prefix=versioned-artifacts/deploy/000001
preview primary artifact version content: b'v1'
preview secondary version on S3: https://console.aws.amazon.com/s3/object/807388292768-us-east-1-artifacts?prefix=versioned-artifacts/deploy/000002
preview secondary artifact version content: b'v2'


In [13]:
ali = repo.put_alias(bsm=bsm, name=name, alias=alias, version=1, secondary_version=2, secondary_version_weight=90)
rprint(ali)

Alias(
    name='deploy',
    alias='LIVE',
    version='1',
    secondary_version='2',
    secondary_version_weight=90,
    version_s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000001',
    secondary_version_s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000002'
)

In [14]:
ali = repo.put_alias(bsm=bsm, name=name, alias=alias, version=2)
rprint(ali)

Alias(
    name='deploy',
    alias='LIVE',
    version='2',
    secondary_version=None,
    secondary_version_weight=None,
    version_s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000002',
    secondary_version_s3uri=None
)

In [15]:
ali_list = repo.list_aliases(bsm=bsm, name=name)
rprint(ali_list)

[
    Alias(
        name='deploy',
        alias='LIVE',
        version='2',
        secondary_version=None,
        secondary_version_weight=None,
        version_s3uri='s3://807388292768-us-east-1-artifacts/versioned-artifacts/deploy/000002',
        secondary_version_s3uri=None
    )
]